In [ ]:
!pip install numpy
!pip install opencv-python
!pip install ultralytics
!pip install supervision

In [ ]:
# Importing libraries
from IPython import display
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
from datetime import timedelta
import os
from IPython.display import HTML
import base64
from IPython.display import Image
display.clear_output()

In [ ]:
def get_video_info(video_path):

    # Extracting information on video
    video_info = sv.VideoInfo.from_video_path(video_path)
    width, height, fps, total_frames = video_info.width, video_info.height, video_info.fps, video_info.total_frames

    # Calculating the length of the video
    video_length = timedelta(seconds = round(total_frames / fps))

    # Print video resolution, fps, and length
    print(f"\033[1mVideo Resolution:\033[0m ({width}, {height})")
    print(f"\033[1mFPS:\033[0m {fps}")
    print(f"\033[1mLength:\033[0m {video_length}")

In [ ]:
# Extracting information of test video
get_video_info('/content/test1.mp4')

Video Resolution: (1280, 720)
FPS: 30
Length: 0:00:31


In [ ]:
!pip install supervision==0.12.0
import supervision as sv

def count_vehicles(video_src, output_folder, line_start_point, line_end_point):

   # Load pre-trained YOLOv8 model
   yolo_model = YOLO('yolov8l.pt')

   #start and end points for the counting line
   start_point = sv.Point(line_start_point[0], line_start_point[1])
   end_point = sv.Point(line_end_point[0], line_end_point[1])

   #zone for line-based vehicle counting
   vehicle_line_counter = sv.LineZone(start_point, end_point)

   # Annotator for the counting line visuals
   line_highlighter = sv.LineZoneAnnotator(thickness = 2,
                                           text_thickness = 1,
                                           text_scale = 0.5)

   #annotator
   box_highlighter = sv.BoxAnnotator(thickness = 1,
                                     text_scale = 0.5)

   # Extract details from the input video
   video_properties = sv.VideoInfo.from_video_path(video_src)

   #the output video path using the source video name
   video_title = os.path.splitext(os.path.basename(video_src))[0] + ".mp4"
   output_video_path = os.path.join(output_folder, video_title)

   #saving the output
   output_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), video_properties.fps, (video_properties.width, video_properties.height))

   #process each frame
   for output_frame in yolo_model.track(source = video_src, tracker = 'bytetrack.yaml', show=True, stream=True, agnostic_nms=True):

        #get the current frame
        current_frame = output_frame.orig_img

        #convert YOLO detection results to Detections object
        detected_objects = sv.Detections.from_yolov8(output_frame)


        if output_frame.boxes.id is not None:
          detected_objects.tracker_id = output_frame.boxes.id.cpu().numpy().astype(int)

        #filter detected objects for cars (class 2) and trucks (class 7)
        detected_objects = detected_objects[(detected_objects.class_id == 2) | (detected_objects.class_id == 7)]

        #labels for tracked objects with class names, tracker IDs, and confidence scores
        detection_labels = [f"{tracker_id} {yolo_model.model.names[class_id]} {confidence:0.2f}"
                            for _, confidence, class_id, tracker_id
                            in detected_objects]

        #trigger vehicle counting for objects intersecting the line zone
        vehicle_line_counter.trigger(detected_objects)

        #draw the counting line and display results
        line_highlighter.annotate(current_frame, vehicle_line_counter)

        #draw bounding boxes and labels for detected vehicles
        current_frame = box_highlighter.annotate(scene = current_frame,
                                                 detections = detected_objects,
                                                 labels = detection_labels)
        #write the modified frame
        output_writer.write(current_frame)


   output_writer.release()
   display.clear_output()


In [ ]:
# Performing vehicle count on the test video
vehicle_count(source_path = "/content/test1.mp4",
              destination_path = "content/drive/MyDrive/VehicleCount/Data/Results",
              line_start = (337, 391),
              line_end = (917, 387))

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.3s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


video 1/1 (frame 1/942) /content/test1.mp4: 384x640 8 cars, 1 bus, 1 truck, 215.6ms


ValueError: too many values to unpack (expected 4)

In [ ]:
#GIF file
gifPath = 'Data/Example Results/test.gif'

# Output the GIF file
Image(filename = gifPath)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Example Results/test1.gif'